In [ ]:
# app_enhanced.py - CS 499 Software Design/Engineering Enhancement
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
import plotly.express as px
import pandas as pd
import base64
import os
from animal_shelter import AnimalShelter

# Load credentials from environment variables
username = os.getenv("MONGO_USER", "aacuser")
password = os.getenv("MONGO_PASS", "SNHU1234")

# Connect to MongoDB
db = AnimalShelter(username, password)
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True, errors='ignore')

# Encode logo image
image_filename = 'grazioso_salvare_logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Create modular layout
def generate_layout():
    return html.Div([
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px'}),
        html.H6("Dashboard by Gonzalo Ramos"),
        html.Hr(),

        html.Div([
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water Rescue'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                    {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
                    {'label': 'Reset', 'value': 'Reset'}
                ],
                value='Reset',
                labelStyle={'display': 'inline-block'}
            )
        ]),
        html.Hr(),

        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            page_size=10,
            style_table={'overflowX': 'auto'},
            style_cell={'textAlign': 'left', 'minWidth': '100px', 'maxWidth': '300px', 'whiteSpace': 'normal'},
            style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
            row_selectable='single',
            sort_action='native',
            filter_action='native'
        ),

        html.Div(className='row', style={'display': 'flex'}, children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ])
    ])

# Initialize app
app = JupyterDash(__name__)
app.layout = generate_layout()

# Callback: Filter table by rescue type
@app.callback(Output('datatable-id', 'data'), [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    rescue_map = {
        'Water Rescue': ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"],
        'Mountain or Wilderness Rescue': ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky"],
        'Disaster Rescue or Individual Tracking': ["German Shepherd", "Doberman Pinscher", "Bloodhound", "Rottweiler"]
    }
    if filter_type == "Reset":
        return df.to_dict('records')
    result = db.read({
        "breed": {"$in": rescue_map[filter_type]},
        "age_upon_outcome_in_weeks": {"$lte": 104}
    })
    fdf = pd.DataFrame.from_records(result)
    fdf.drop(columns=['_id'], inplace=True, errors='ignore')
    return fdf.to_dict('records')

# Callback: Update pie chart
@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData) if viewData else df
    return [dcc.Graph(figure=px.pie(dff, names='breed', title='Preferred Animals'))]

# Callback: Highlight selected column
@app.callback(Output('datatable-id', 'style_data_conditional'), [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

# Callback: Show geolocation map for selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if not viewData or not index:
        return html.Div()
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run the server
app.run_server(debug=True)


Dash app running on http://127.0.0.1:19257/
